## Streaming Real Time Options Data with ThetaData

In [ ]:
import datetime as dt
from thetadata import (
    Quote,
    StreamMsg,
    ThetaClient,
    OptionRight,
    StreamMsgType,
    StreamResponseType,
)

Initialize global variables for last quotes and price

In [ ]:
last_call_quote = Quote()
last_put_quote = Quote()
price = 0

In [ ]:
def callback_straddle(msg):
    # Handle received messages
    if msg.type != StreamMsgType.QUOTE:
        return

    global price

    # Update the last call or put quote based on the message
    if msg.contract.isCall:
        last_call_quote.copy_from(msg.quote)
    else:
        last_put_quote.copy_from(msg.quote)

    # Calculate straddle bid, ask, and mid prices
    straddle_bid = round(last_call_quote.bid_price + last_put_quote.bid_price, 2)
    straddle_ask = round(last_call_quote.ask_price + last_put_quote.ask_price, 2)
    straddle_mid = round((straddle_bid + straddle_ask) / 2, 2)

    # Get the timestamp from the message
    time_stamp = thetadata.client.ms_to_time(msg.quote.ms_of_day)

    # Print the straddle prices if they have changed
    if price != straddle_mid:
        print(
            f"time: {time_stamp} bid: {straddle_bid} mid: {straddle_mid} ask: {straddle_ask}"
        )
        price = straddle_mid

In [ ]:
def stream_straddle():
    # Initialize the ThetaClient with your credentials
    client = ThetaClient(username="YOUR_EMAIL", passwd="YOUR_PASSWORD")
    
    # Connect to the data stream
    client.connect_stream(callback_straddle)
    
    # Request quote streams for call and put options
    req_id_call = client.req_quote_stream_opt(
        "SPY", dt.date(2024, 3, 28), 475, OptionRight.CALL
    )
    req_id_put = client.req_quote_stream_opt(
        "SPY", dt.date(2024, 3, 28), 475, OptionRight.PUT
    )
    
    # Verify the subscriptions
    if (
        client.verify(req_id_call) != StreamResponseType.SUBSCRIBED
        or client.verify(req_id_put) != StreamResponseType.SUBSCRIBED
    ):
        raise Exception("Unable to stream.")

Start streaming straddle quotes

In [ ]:
stream_straddle()

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.